In [ ]:
import pandas as pd
from transformers import pipeline
from huggingface_hub import notebook_login

notebook_login()

Import loading and login process into hugging face for model authorization.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
def format_prompt(text, task):
    if task == "orientation":
        return f"Classify the following speech as belonging to a left-wing (label 0) or right-wing (label 1) politician: {text}"
    elif task == "power":
        return f"Determine whether the following speech is from a governing party (label 0) or an opposition party (label 1): {text}"
    else:
        raise ValueError("Invalid task. Choose 'orientation' or 'power'.")

model_name = "meta-llama/Llama-3.2-1B"
classifier = pipeline("text-classification", model=model_name, tokenizer=model_name, device=0)

test_data_path = "/kaggle/input/power-dataset/power-tr-train.tsv"
task = "power"

print("Loading test data...")
test_data = pd.read_csv(test_data_path, sep='\t')
test_data = test_data.dropna(subset=["text_en", "label"])

test_data = test_data.head(500)

print("Running zero-shot inference...")
predictions = []
for index, row in test_data.iterrows():
    prompt = format_prompt(row['text_en'], task)
    result = classifier(prompt)
    predicted_label = result[0]['label']
    predictions.append(predicted_label)

true_labels = test_data['label'].tolist()
predicted_labels = [int(label[-1]) for label in predictions]

accuracy = accuracy_score(true_labels, predicted_labels)
precision, recall, f1, _ = precision_recall_fscore_support(
    true_labels, predicted_labels, average='binary', zero_division=0
)

print(f"Zero-shot inference results for {task} task:")
print(f"Accuracy: {accuracy:.2%}")
print(f"Precision: {precision:.2%}")
print(f"Recall: {recall:.2%}")
print(f"F1 Score: {f1:.2%}")

### Zero-Shot Inference Flow - English
- Inference flow with English prompts.
- File path and task type can be changed to variate between power and orientation tasks.